<a href="https://colab.research.google.com/github/ostrokach/beam-notebooks/blob/master/notebooks/pubsub_streaming_pull.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install dependencies

In [ ]:
try:
    from google.colab import auth
    GOOGLE_COLAB = True
except ImportError:
    GOOGLE_COLAB = False

In [ ]:
if GOOGLE_COLAB:
    !pip install google-cloud-pubsub

## Imports

In [ ]:
from google.cloud import pubsub
from google.api_core import exceptions as gexc

## Parameters

In [ ]:
# { display-mode: "form" }
project_id = "strokach-playground" #@param {type:"string"}

In [ ]:
if GOOGLE_COLAB:
    auth.authenticate_user()

## Workflow

### Following the documentation

In [ ]:
sub_client = pubsub.SubscriberClient()

In [ ]:
subscription_name = "projects/{}/subscriptions/taxirides-realtime-sub".format(project_id)

try:
    sub_client.create_subscription(
        subscription_name,
        "projects/pubsub-public-data/topics/taxirides-realtime",
    )
except gexc.AlreadyExists:
    sub_client.delete_subscription(subscription_name)
    sub_client.create_subscription(
        subscription_name,
        "projects/pubsub-public-data/topics/taxirides-realtime",
    )

In [ ]:
requests = [{"subscription": subscription_name, "stream_ack_deadline_seconds": 10}]

In [ ]:
for i, response in enumerate(sub_client.streaming_pull(requests=requests, timeout=10)):
    print(response)
    if i >= 10:
        break

### Something that seems to work

In [ ]:
sub_client = pubsub.SubscriberClient()

In [ ]:
subscription_name = "projects/{}/subscriptions/taxirides-realtime-sub".format(project_id)

try:
    sub_client.create_subscription(
        subscription_name,
        "projects/pubsub-public-data/topics/taxirides-realtime",
    )
except gexc.AlreadyExists:
    sub_client.delete_subscription(subscription_name)
    sub_client.create_subscription(
        subscription_name,
        "projects/pubsub-public-data/topics/taxirides-realtime",
    )

In [ ]:
def request_generator():
    r = pubsub.types.StreamingPullRequest()
    r.subscription = subscription_name
    r.stream_ack_deadline_seconds = 10
    yield r

    while True:
        r = pubsub.types.StreamingPullRequest()
        yield r

In [ ]:
for i, response in enumerate(sub_client.streaming_pull(requests=request_generator(), timeout=10)):
    print(response)
    if i >= 10:
        break